# Currently missing things (will need testing)
- New packages (those would have a `File.status` of `created` iirc)
- Renamed packages (those would be [I think] like the above but with a status of `renamed` on the link)
- Deleted packages (I think that would be cleanly all templates that go `deleted`)

In [113]:
import github
from collections import defaultdict
import pathlib

In [114]:
with pathlib.Path("./tokenfile").open() as tokenfile:
    token = tokenfile.readline().strip()

In [115]:
gh = github.Github(login_or_token=token)
repo = gh.get_repo("void-linux/void-packages")

# the PR number goes here
pr = repo.get_pull(36376)

files = list(pr.get_files())

In [116]:
def ischange(line: str):
    return line.startswith(("+", "-"))
def patch_changes(patch: str):
    return list(filter(ischange, patch.splitlines()))

In [117]:
patch_changes(files[0].patch)

['-version=15.0.3',
 '+version=15.1.0',
 '- 7c6a4d8e003cab1a46f86fdf43f8c49e4b9d6ce932961572476266ad9f351608',
 '- 629dd3cf957f7ef255e7dccc3286c5a3b54c0a6a7cde8b93b3a7fdc9dffc4cbd"',
 '+ 4bb10d12681546fa129f9105706d6f55c346b75418a19793fb300f0bd4e5991d',
 '+ e05995a4b66592d5868b7e667bba8a5f1822499fc53c6cd7a8abf3e40f8b8895"']

In [118]:
_labels = []
_pkgs: dict[str, list] = defaultdict(lambda: list())
for file in files:
    fn = file.filename
    if fn.startswith(("xbps-src", "common/", "etc/")):
        _labels += ["xbps-src"]
        continue
    if fn in ("Manual.md", "README.md", "CONTRIBUTING.md"):
        _labels += ["documentation"]
        continue
    if fn.startswith(("srcpkgs/")):
        # TODO: DEAL WITH NEW PACKAGES
        fn = fn.removeprefix("srcpkgs/")
        fp = fn.split("/")
        pkg = fp[0]
        if len(fp) >= 2 and (fp[1] == "template"):
            changes = patch_changes(file.patch)
            for chg in changes:
                if chg.startswith(("-version=", "+version=")):
                    _pkgs[pkg] += ["update"]
                    continue
                if chg.startswith(("-revision=", "+revision=")):
                    _pkgs[pkg] += ["revbump"]
                else:
                    _pkgs[pkg] += ["cleanup-fixups"]
        else:
            _pkgs[pkg] += ["cleanup-fixups"]

# Give preference 'update', since an update strongly implies
# a rev reset and cleanups/fixups for compilation
for p, l in _pkgs.items():
    if "update" in l:
        _labels += ["update"]
    else:
        _labels += l
_labels

['update']

### The labels to apply to the PR
The set deduplicates the lebels for us

In [119]:
set(_labels)

{'update'}